## **앱스토어 리뷰 감성 분석**

### 문제정의
각 앱들의 리뷰들과 평점이 양의 관계에 있다고 가정하고 학습을 진행하여 평점을 예측

### 데이터 불러오기

tsv 파일 불러오기

In [6]:
def get_data_split(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f.readlOOines()]
        docs = [doc for doc in docs if len(doc) == 3]
        
    if not docs:
        app_ids, texts, ratings = [], [], []

    app_ids, texts, ratings = zip(*docs)
    return list(app_ids), list(texts), list(ratings)

In [3]:
train_app_ids, train_texts, train_ratings = get_data_split('./data/train_docs.txt')
test_app_ids, test_texts, test_ratings = get_data_split('./data/test_docs.txt')

json 파일 불러오기

In [1]:
from collections import Counter, defaultdict
from konlpy.tag import Twitter, Komoran, Mecab
from utils import Unknown_words, Post_ma, Make_dictionay
import ujson

twitter = Twitter()
komoran = Komoran()
mecab = Mecab()

def get_data_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        texts = file.read()
        data = ujson.loads(texts)
        app_ids_list = list(data.keys())
    return data, app_ids_list

In [55]:
# train_data, train_app_id_list = get_data_json('./data/train_json.txt')
# test_data, test_app_id_list = get_data_json('./data/test_json.txt')

### 데이터 전처리: 형태소 분석
1. 축약어, 오타:
골치거리임, 전형적인 한국 사전에 없는 단어들도 많음, 사전에 없는 단어를 트위터사전에 넣어야 할 듯
2. 띄어쓰기가 불분명함

$\Rightarrow$ data Cleaning class 필요성 있음

### 데이터 후처리

데이터 후처리란 형태소 분석기를 거친 후에 이상한 단어들, 신조어 등을 처리하는 작업


각 앱별로 공백문자를 나눠서 공통된 단어를 추출해보기? 앱별로 공통으로 많이 이야기 하는 단어들이 제대로 형태소 분석이 되는지 테스트 하기 위해 단어들을 추출해본다

문제점: 일일이 확인 하기 어려움

## 형태소 분석 후처리

---

## komoran
이게 제일 나은듯?


In [3]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_komoran.txt')

'NA'확인 결과 문장 자체가 분석이 안된 것이 주로 ㅜㅜ 등 모음이나 자음이 따로 있는 경우 였다. 이것 부터 해결해보자

In [4]:
UNK = Unknown_words(train_ma_docs, app_id_list)

In [5]:
unknown_list = UNK.get_unknown_words()

Unknown: 70342, total: 173157


In [6]:
UNK.unknown_loc['된다고하네요ㅜㅜ환불해주세요']

[['1021117809', (51, 11)]]

In [7]:
train_ma_docs['1021117809'][51]

[['이것', 'NP'],
 ['무료', 'NNG'],
 ['앱', 'NNP'],
 ['인줄', 'NNG'],
 ['알', 'VV'],
 ['았', 'EP'],
 ['는데', 'EC'],
 ['사람', 'NNG'],
 ['들', 'XSN'],
 ['이', 'JKS'],
 ['결제', 'NNG'],
 ['된다고하네요ㅜㅜ환불해주세요', 'NA']]

In [8]:
unknown_rule, rest_unknown_list = UNK.remove_no_meanings(unknown_list, komoran)
len(rest_unknown_list)

39287

리스트 형태로 바꿔줌

In [9]:
post_ma_list = UNK.shape_changer(unknown_rule)

형태소 분석기 돌리기 전에 우선 문법을 좀 고쳐야 되는ㄱ ㅓ아닌가 ..됬는데 --> 됐는데...

우선 단어 사전을 만들고 다시 unknown에 대해서 단어가 있으면 스페이싱을 하는 방법?

혹은 자음모음 나눠서 단어장에 유사도가 비슷하면 채택?

In [10]:
post_ma = Post_ma(tokenizer_name='komoran')

Load Complete! total 31055 rules


In [14]:
post_ma.update(post_ma_list, write=True)

In [18]:
post_ma.post_ma_pairs[0]

([['만원추가ㅡㅡ', 'NA']], [['만원', 'NNG'], ['추가', 'NNG']])

In [12]:
len(UNK.unknown_loc)

70342

In [13]:
train_ma_docs = post_ma.replace_ma_docs(train_ma_docs, UNK.unknown_loc)

스위칭 됬는지 확인!

In [14]:
train_ma_docs['1021117809'][51]

[['이것', 'NP'],
 ['무료', 'NNG'],
 ['앱', 'NNP'],
 ['인줄', 'NNG'],
 ['알', 'VV'],
 ['았', 'EP'],
 ['는데', 'EC'],
 ['사람', 'NNG'],
 ['들', 'XSN'],
 ['이', 'JKS'],
 ['결제', 'NNG'],
 ['되', 'VV'],
 ['ㄴ다고', 'EC'],
 ['하', 'VX'],
 ['네요', 'EC'],
 ['환불', 'NNG'],
 ['하', 'XSV'],
 ['아', 'EC'],
 ['주', 'VX'],
 ['시', 'EP'],
 ['어요', 'EC']]

한번 다시 반복

In [15]:
UNK = Unknown_words(train_ma_docs, app_id_list)
unknown_list = UNK.get_unknown_words()

Unknown: 42020, total: 133148


새로 한번 적용했으나 이제 거의 여과가 안됨 아까는 70342 $\rightarrow$ 39287 로 줄었는데, 지금은 42020 $\rightarrow$ 39272 로 줄음

In [16]:
unknown_rule, rest_unknown_list = UNK.remove_no_meanings(unknown_list, komoran)
len(rest_unknown_list)

39272

In [17]:
post_ma_list = UNK.shape_changer(unknown_rule)

In [19]:
len(post_ma_list)

2748

이제 새로나온 룰을 추가시킨다

In [20]:
post_ma.update(post_ma_list)

In [22]:
post_ma.load()

In [23]:
len(post_ma.post_ma_pairs)

33803

나머지는 단어 오탈자, 문장끝에 '닼ㅋㅋ' 이런 것들이 주로 많은 편이다, 이건 좀 나중에...

In [25]:
rest_unknown_list[:20]

['팝업땨문에',
 '띵똥으로',
 '소맄ㅋ',
 '별로거든욯ㅎ~~^^',
 '간간히뒤에',
 '아둥바둥하는듭!',
 '받았는뎈ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '안되네요...쩝',
 '버벅거려..',
 '솔로탈춯함',
 '돈벋아먹을건',
 '구매내역삭제부탁드령용',
 '븅딱새끼들이.',
 '덜된건짛ㅎ',
 '바깟었는데',
 '딱이예욤!',
 '열어봐욯ㅎ',
 '인식됬으면',
 '페메에',
 '보정가능합니닼ㅋㅋ']

마지막 체크

In [26]:
UNK = Unknown_words(train_ma_docs, app_id_list)
unknown_list = UNK.get_unknown_words()

Unknown: 42020, total: 133148


저장하기

In [28]:
def save_ma_as_json(filename, ma_docs):
    with open(filename, 'w', encoding='utf-8') as out_file:
        data = ujson.dumps(ma_docs, ensure_ascii=False)
        print(data, file=out_file)

In [29]:
save_ma_as_json('./data/ma_train_json_komoran_after.txt', train_ma_docs)

---

## Twitter

트위터가 생각보다 별로 인듯...

In [9]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_twitter.txt')

---

### 주요 품사별 추출 및 사전 만들기

MNG(일반 명사), NNP(고유 명사), VV(동사), VA(형용사)이며, 경우에 따라 MM(관형사), MAG(일반 부사), MAJ(접속 부사)

일단 노 추출하고 사전 만들어보자

In [1]:
from collections import Counter, defaultdict
from konlpy.tag import Twitter, Komoran, Mecab
from utils import Unknown_words, Post_ma, Make_dictionay
import ujson

twitter = Twitter()
komoran = Komoran()
mecab = Mecab()

def get_data_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        texts = file.read()
        data = ujson.loads(texts)
        app_ids_list = list(data.keys())
    return data, app_ids_list

In [2]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_komoran_after.txt')

In [18]:
train_data, _ = get_data_json('./data/train_json.txt')
train_ratings = {k: v['ratings'] for k, v in train_data.items()}
del train_data
del _

In [3]:
make_dict = Make_dictionay()

In [4]:
make_dict.fit(train_ma_docs, maxwords=100000)

total words: 100000
complete!


In [6]:
train_ma_docs_idx = make_dict.get_ma_docs_idx(train_ma_docs)

In [27]:
train_ma_docs[app_id_list[436]][1]

[['별', 'MM'],
 ['10', 'SN'],
 ['개', 'NNB'],
 ['드리', 'VV'],
 ['ㅂ니다', 'EC'],
 ['~', 'SO']]

In [28]:
train_ma_docs_idx[app_id_list[436]][1]

[394, 359, 154, 47, 34, 35]

In [30]:
# inverse dict: 편리하게 단어를 찾을 수 있음 
make_dict.word_dict_idx_inverse[47]

'드리/VV'

In [77]:
make_dict.word_dict_idx_inverse[88888]

'IP부띠끄호텔에서/NA'

## Navie Bayesian Classifier

한 documentation으로 보면, Input A은 문장내에 단어들의 조합이다 $[a_1, a_2 ,\cdots, a_n]$. B는 rating점수다. 각 rating점수에 대해서, 

$P(B|A) = \frac{P(A|B)P(B)}{P(A)}$가 성립하는데, $P(B)$는 사전 확률로 문서내에 각 rating이 나온 횟수로 측정 , $P(A|B)$는 Likelihood다.

Chain rule를 활용하면 Likelihood $\times$ prior 은 아래와 같이 된다.

$P(a_1, a_2 ,\cdots, a_n, B) \\ 
= P(a_1, a_2 ,\cdots, a_n|B)P(B) \\
= P(a_2, \cdots, a_n|B, a_1)P(a_1|B)P(B) \\
= \cdots = P(a_n|B, a_1, \cdots, a_{n-1}) \cdots P(a_1|B)P(B)$

여기서 $P(B)$를 제외한 부분은 구하기 어렵기 때문에 Navie가정을 하여($a_i$는 각기 다른 $a_i$과 독립이며, B에 대하여 조건부 독립이다) 계산. 즉,

$P(a_1|B) = P(a_1|B)\\
P(a_2|B, a_1) = P(a_2|B)\\
\quad \vdots \\
P(a_n|B, a_1, \cdots, a_{n-1}) = P(a_n|B)$

가 되는 것이고 종합하게 되면 $ P(a_1, a_2 ,\cdots, a_n, B) = \prod_{i=1}^{n}{P(a_i|B)}P(B)$ 가 된다.

In [ ]:
rating_counter = Counter()
for app_id in app_id_list:
    rating_counter
    

### td-idf

## Doc2vec

In [20]:
from gensim.models import Doc2Vec, Word2Vec

In [ ]:
Word2Vec()